In [2]:
# Imports
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv('../Day19/train.csv')

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Plan

- Impute missing values - Age & Embarked
- One Hot Encode - Sex & Embarked
- Scaling
- Feature selection
- Train decision tree model

In [6]:
# Drop useless columns
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [7]:
# Train test split
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), 
                                                    df['Survived'], 
                                                    test_size=0.2,
                                                    random_state=42
                                                    )

In [8]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [9]:
# Imputation transformer
trf1 = ColumnTransformer([
    ('Impute Age', SimpleImputer(), [2]),
    ('Impute Embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough')

In [10]:
# One Hot Encoding transformer
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
], remainder='passthrough')

In [11]:
# Scaling
trf3 = ColumnTransformer([
    ('Scale', MinMaxScaler(), slice(0,10))
])

In [12]:
# Feature selection
trf4 = SelectKBest(score_func=chi2, k = 8)

In [13]:
# Train the model
trf5 = DecisionTreeClassifier()

## Create pipeline

In [14]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5),
])

## Pipeline vs make_pipeline

- Pipeline requires naming of steps, make_pipeline does not
- Same applies to ColumnTransfer vs make_column_transformer

In [15]:
# Alternate syntax
pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

- If you're not training model at the end of the pipeline, call fit_transform, so that you don't have to manually fit and then call transform
- If you're training the model at the end of the pipeline, call fit, so that you're able to use predict

In [16]:
pipe.fit(x_train, y_train)

,steps,"[('columntransformer-1', ...), ('columntransformer-2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('Impute Age', ...), ('Impute Embarked', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [17]:
# check the steps of the pipeline
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('Impute Age', SimpleImputer(), [2]),
                                 ('Impute Embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('Scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x761c61f02b60>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [18]:
# deep dive into individual steps
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.49884615])

In [19]:
# Predict
y_pred = pipe.predict(x_test)

In [20]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [21]:
# Determine the accuracy score
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.6256983240223464

## Cross validation using pipeline

In [22]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipe, x_train, y_train, cv=5, scoring='accuracy').mean()

np.float64(0.6391214419383433)

## Grid search using pipeline

In [27]:
# GridsearchCV
params = {
    'decisiontreeclassifier__max_depth': [1,2,3,4,5,None]
}

In [28]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, params, cv = 5, scoring='accuracy')
grid.fit(x_train, y_train)

,estimator,Pipeline(step...lassifier())])
,param_grid,"{'decisiontreeclassifier__max_depth': [1, 2, ...]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('Impute Age', ...), ('Impute Embarked', ...)]"


In [29]:
grid.best_score_

np.float64(0.6391214419383433)

In [30]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 2}

## Exporting the pipeline

In [31]:
import pickle

pickle.dump(pipe, open('pipe.pkl', 'wb'))